# Agentic Retrieval Quickstart for Azure AI Search

### 1. Load Connections

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
answer_model = os.getenv("ANSWER_MODEL", "gpt-4o")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create search agent

In [2]:
import requests

create_agent_request = {
    "name": agent_name,
    "targetIndexes": [ { "indexName": index_name } ],
    "models": [
          {
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "resourceUri": azure_openai_endpoint,
                "apiKey": None,
                "deploymentId": azure_openai_gpt_model,
                "modelName": azure_openai_gpt_model
            }
        }
    ]
}

response = requests.put(
    url=f"{endpoint}/agents/{agent_name}?api-version={api_version}",
    headers={ "Authorization": f"Bearer {token_provider()}" },
    json=create_agent_request
)
response.raise_for_status()


### 3. Setup messages

In [3]:
instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

### 4. Use Agentic Retrieval to fetch results

In [4]:
def query_agent(messages: list[dict[str, any]]):
    retrieval_request = {
        "messages" : [ { "role": msg["role"], "content": [ { "text": msg["content"] , "type": "text" } ] } for msg in messages ],
        "targetIndexParams" :  [
            { 
                "indexName" : index_name,
                "rerankerThreshold": 2.5
            } 
        ]
    }
    response = requests.post(
        url=f"{endpoint}/agents/{agent_name}/retrieve?api-version={api_version}",
        headers={ "Authorization": f"Bearer {token_provider()}" },
        json=retrieval_request
    )
    response.raise_for_status()
    result = response.json()
    return result

In [5]:
messages.append({
    "role": "user",
    "content": """
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
})

retrieval_result = query_agent(messages)
messages.append({
    "role": "assistant",
    "content": retrieval_result["response"][0]["content"][0]["text"]
})

### 4.1. Review retrieval activity and results

In [6]:
import textwrap

print("Response")
print(textwrap.fill(retrieval_result["response"][0]["content"][0]["text"], width=120))

Response
[{"ref_id":1,"content":"# Urban Structure\n\n## March 16, 2013\n\n### Phoenix Metropolitan Area at Night\n\nThis figure
presents a nighttime satellite view of the Phoenix metropolitan area, highlighting urban structure and transport
corridors. City lights illuminate the layout of several cities and major thoroughfares.\n\n**Labeled Urban
Features:**\n\n- **Phoenix:** Central and brightest area in the right-center of the image.\n- **Glendale:** Located to
the west of Phoenix, this city is also brightly lit.\n- **Peoria:** Further northwest, this area is labeled and its
illuminated grid is seen.\n- **Grand Avenue:** Clearly visible as a diagonal, brightly lit thoroughfare running from
Phoenix through Glendale and Peoria.\n- **Salt River Channel:** Identified in the southeast portion, running through
illuminated sections.\n- **Phoenix Mountains:** Dark, undeveloped region to the northeast of Phoenix.\n- **Agricultural
Fields:** Southwestern corner of the image, grid patterns are 

In [7]:
import json
print("Activity")
print(json.dumps(retrieval_result["activity"], indent=2))
print("Results")
print(json.dumps(retrieval_result["references"], indent=2))

Activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 1407,
    "outputTokens": 523
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening compared to urban cores",
      "filter": null
    },
    "queryTime": "2025-04-30T16:03:34.508Z",
    "elapsedMs": 625
  },
  {
    "type": "AzureSearchQuery",
    "id": 2,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space",
      "filter": null
    },
    "queryTime": "2025-04-30T16:03:34.869Z",
    "count": 2,
    "elapsedMs": 361
  },
  {
    "type": "AzureSearchQuery",
    "id": 3,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "dim interstates between midwestern cities at night",
      "filter": null
    },
    "queryTime": "2025-04-30T16:03:35.174Z",
    "elapsedMs": 304
  }
]
Results
[
  {
    "type": "AzureSearchDoc",
   

### 5. Create Azure OpenAI Client

In [8]:
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    azure_ad_token_provider=azure_openai_token_provider,
    api_version=azure_openai_api_version
)

### 5.1 Use Responses API to generate an answer

In [9]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

The larger December brightening in suburban belts compared to urban cores, despite higher absolute
light levels downtown, can be attributed to the widespread distribution of decorative and seasonal
lighting in suburban areas. Suburbs have more residential properties where such lighting is
prevalent and can create a greater relative increase in brightness compared to urban centers with
more commercial or industrial lighting that remains constant [0].  As for the Phoenix nighttime
street grid being sharply visible from space, this is due to its regular grid pattern of city blocks
and streets, which are extensively lit. Major streets and intersections in the Phoenix metropolitan
area are designed for optimal light distribution, enhancing their visibility at night from space. In
contrast, large stretches of interstate highways between midwestern cities are less densely
populated and have fewer adjacent lit areas, resulting in dimmer illumination levels [0][1].


### 5.2 Use Chat Completions API to generate an answer

In [10]:
response = client.chat.completions.create(
    model=answer_model,
    messages=messages
)

wrapped = textwrap.fill(response.choices[0].message.content, width=100)
print(wrapped)

Suburban belts exhibit greater December brightening than urban cores mainly due to the distribution
and type of holiday lighting and decorations prevalent in suburban areas. Suburbs often have larger
residential properties and open spaces that accommodate extensive outdoor holiday lighting, leading
to noticeable brightening during December. In contrast, urban cores have higher absolute light
levels year-round due to concentrated commercial and industrial activities, which do not change
significantly with seasonal lighting displays.  The Phoenix nighttime street grid is sharply visible
from space because it follows a structured, regular grid pattern that is typical of many western
U.S. cities. This layout is enhanced by extensive street lighting, making it distinct when viewed
from the low-Earth-orbit vantage point of satellites or the International Space Station. In
comparison, large stretches of the interstate between midwestern cities are less illuminated, often
traversing rural, les

### 6. Continue the conversation

In [11]:
messages.append({
    "role": "user",
    "content": "How do I find lava at night?"
})

retrieval_result = query_agent(messages)
messages.append({
    "role": "assistant",
    "content": retrieval_result["response"][0]["content"][0]["text"]
})

### 6.1. Review activity and results

In [12]:
print("Response")
print(textwrap.fill(retrieval_result["response"][0]["content"][0]["text"], width=120))

Response
[{"ref_id":1,"content":"For the first time in perhaps a decade, Mount Etna experienced a \"flank eruption\"—erupting
from its side instead of its summit—on December 24, 2018. The activity was accompanied by 130 earthquakes occurring over
three hours that morning. Mount Etna, Europe’s most active volcano, has seen periodic activity on this part of the
mountain since 2013. The Operational Land Imager (OLI) on the Landsat 8 satellite acquired the main image of Mount Etna
on December 28, 2018.\n\nThe inset image highlights the active vent and thermal infrared signature from lava flows,
which can be seen near the newly formed fissure on the southeastern side of the volcano. The inset was created with data
from OLI and the Thermal Infrared Sensor (TIRS) on Landsat 8. Ash spewing from the fissure cloaked adjacent villages and
delayed aircraft from landing at the nearby Catania airport. Earthquakes occurred in the subsequent days after the
initial eruption and displaced hundreds of pe

In [13]:
print("Activity")
print(json.dumps(retrieval_result["activity"], indent=2))
print("Results")
print(json.dumps(retrieval_result["references"], indent=2))

Activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 2283,
    "outputTokens": 108
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "detecting lava at night",
      "filter": null
    },
    "queryTime": "2025-04-30T16:03:46.940Z",
    "count": 2,
    "elapsedMs": 370
  }
]
Results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_44_verbalized",
    "sourceData": null
  },
  {
    "type": "AzureSearchDoc",
    "id": "1",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_46_verbalized",
    "sourceData": null
  }
]


### 6.2. Generate answer

In [14]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

To find lava at night, you should look for glowing lava flows from active volcanoes. These can often
be seen due to the intense heat and light they emit. Using satellite imagery can help track these
flows. Instruments like the VIIRS Day/Night Band on polar-orbiting satellites can detect faint light
sources and thermal infrared signatures of lava flows, even during nighttime. Additionally, the glow
from lava is often visible to the naked eye, but for safety, it's best monitored from a distance or
via technology. [ref_id: 0, 1]
